# CaloriePhone: food-to-calorie translation

TODO description

### Import libraries


In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

import getpass

In [2]:
tf.__version__

'2.8.0'

In [3]:
# InceptionResNetV2 accepts 299x299 images as input
IMG_SIZE = (299, 299)
IMG_SHAPE = IMG_SIZE + (3,)
MOUNT_PATH = '/mnt/drive'
DATA_DIR = '/mnt/drive/MyDrive/caloriephone/data'

# mount drive
from google.colab import drive
drive.mount(MOUNT_PATH)

Mounted at /mnt/drive


### Data Preprocessing


Besides building the classificator, we need to know how many calories each kind of food contains. We will be using the `vaishnavivenkatesan/food-and-their-calories` dataset which is a CSV file of different foods and the appropriate amount of calories in a single serving of each.

In [4]:
# we will need to download the mapping from kaggle datasets
# let's make sure kaggle cli is installed
!python3 -m pip install kaggle

# get log-in credentials
# get credentials from https://www.kaggle.com -> click on your username on the right -> "Account" in the middle bar -> "Create New API Token" 
os.environ["KAGGLE_USERNAME"] = input("Kaggle username: ")
os.environ["KAGGLE_KEY"] = getpass.getpass("Kaggle key: " )

# download mapping
!kaggle datasets download --unzip vaishnavivenkatesan/food-and-their-calories

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Kaggle username: vladpoberezhny
Kaggle key: ··········
  0% 0.00/6.59k [00:00<?, ?B/s]
100% 6.59k/6.59k [00:00<00:00, 5.38MB/s]


In [5]:
# read food to calorie mapping CSV
mapping_csv = pd.read_csv("Food and Calories - Sheet1.csv")
mapping_csv.head()

,Food,Serving,Calories
0,Artichoke,1 artichoke (128 g),60 cal
1,Arugula,1 leaf (2 g),1 cal
2,Asparagus,1 spear (12 g),2 cal
3,Aubergine,1 aubergine (458 g),115 cal
4,Beetroot,1 beet (82 g),35 cal


Let's see how many food types there are in this CSV:

In [6]:
# each food type is a row
# thus amount of rows is indicative of the amount of possible classes
CLASSES_AMOUNT = mapping_csv.shape[0]
print(CLASSES_AMOUNT)

562


Some of the classes found in this CSV are extremely specific. They include classes such as:

- Big N' Tasty (commercial name for a McDonald's burger)
- Burger King Angry Whopper (also a commercial name, not an actual food type)
- Alphabet Soup	(a soup that has latter shaped pasta in it)

We aim to classify different food types, as opposed to specific foods, so we have decided to filter out some of the classes.

In [7]:
# define a blacklist of foods
food_class_blacklist = [
  "Bacon and Eggs",
  "Black Pudding",
  "BLT",
  "California Roll",
  "Chicken Marsala",
  "Chicken Parmesan",
  "Chicken Pot Pie",
  "Chicken Tikka Masala",
  "Cobb Salad",
  "Corned Beef Hash",
  "Cottage Pie",
  "Deviled Eggs",
  "Dim Sum",
  "Macaroni and Cheese",
  "Orange Chicken",
  "Philly Cheese Steak",
  "Pulled Pork Sandwich",
  "Reuben Sandwich",
  "Roast Beef",
  "Roast Dinner",
  "Sausage Rolls",
  "Shrimp Cocktail",
  "Sloppy Joe",
  "Sloppy Joes",
  "Spaghetti Bolognese",
  "Spring Rolls",
  "Tandoori Chicken",
  "Yorkshire Pudding",
  "Durum Wheat Semolina",
  "Wheat Bran",
  "Wheat Germ",
  "Wheat Gluten",
  "Wheat Semolina",
  "Wheat Starch",
  "Whole Grain Wheat",
  "Wholegrain Oat",
  "Apricot Kernel Oil",
  "Argan Oil",
  "Babassu Oil",
  "Cottonseed Oil",
  "Flaxseed Oil",
  "Grape Seed Oil",
  "Hazelnut Oil",
  "Linseed Oil",
  "Menhaden Oil",
  "Mustard Oil",
  "Oat Oil",
  "Palm Kernel Oil",
  "Palm Oil",
  "Peanut Oil",
  "Poppy Seed Oil",
  "Pumpkin Seed Oil",
  "Rice Bran Oil",
  "Safflower Oil",
  "Salmon Oil",
  "Sesame Oil",
  "Shea Oil",
  "Soy Oil",
  "Sunflower Oil",
  "Tomato Seed Oil",
  "Vegetable Oil",
  "Walnut Oil",
  "Wheat Germ Oil",
  "Alphabet Soup",
  "Beef Noodle Soup",
  "Broccoli Cheese Soup",
  "Carrot Ginger Soup",
  "Chicken Gumbo Soup",
  "Chicken Noodle Soup",
  "Chicken Vegetable Soup",
  "Chicken with Rice Soup",
  "Cream of Asparagus Soup",
  "Cream of Broccoli Soup",
  "Cream of Celery Soup",
  "Cream of Chicken Soup",
  "Cream of Mushroom Soup",
  "Cream of Onion Soup",
  "Cream of Potato Soup",
  "Creamy Chicken Noodle Soup",
  "Golden Mushroom Soup",
  "Lobster Bisque Soup",
  "Meatball Soup",
  "Oxtail Soup",
  "Potato Soup",
  "Pumpkin Soup",
  "Scotch Broth",
  "Thai Soup",
  "Tomato Rice Soup",
  "Tomato Soup",
  "Vegetable Beef Soup",
  "Vegetable Broth",
  "Vegetable Stock",
  "Wedding Soup",
  "Butter Pecan Ice Cream",
  "Chocolate Chip Ice Cream",
  "Ciao Bella",
  "Coffee Ice Cream",
  "Cold Stone Creamery",
  "Cookie Dough Ice Cream",
  "Crunchie McFlurry",
  "Dairy Milk McFlurry",
  "Dippin Dots",
  "Double Rainbow",
  "Drumsticks",
  "French Vanilla Ice Cream",
  "Friendly’s",
  "Healthy Choice",
  "Hot Fudge Sundae",
  "Ice Cream Sandwich",
  "Ice Cream Sundae",
  "Ice Milk",
  "Magnolia",
  "Magnum",
  "Magnum Almond",
  "Magnum Double Caramel",
  "Magnum Double Chocolate",
  "Magnum Gold",
  "Magnum White",
  "McFlurry",
  "McFlurry Oreo",
  "Mini Milk",
  "Mint Chocolate Chip Ice Cream",
  "Rocky Road Ice Cream",
  "Schwan’s",
  "Smarties McFlurry",
  "Snickers Ice Cream",
  "Soft Serve",
  "Strawberry Ice Cream",
  "Strawberry Sundae",
  "Turkey Hill",
  "Vanilla Cone",
  "Vanilla Ice Cream",
  "Cellophane Noodles",
  "Cheese Tortellini",
  "Dampfnudel",
  "Dumpling Dough",
  "Durum Wheat Semolina",
  "Glass Noodles",
  "Lasagne Sheets",
  "Low Carb Pasta",
  "Shirataki Noodles",
  "Soy Noodles",
  "Spinach Tortellini",
  "Whole Grain Noodles",
  "Whole Grain Spaghetti",
  "BBQ Chicken Pizza",
  "BBQ Pizza",
  "Beef Pizza",
  "Bianca Pizza",
  "Buffalo Chicken Pizza",
  "Calabrese Pizza",
  "Calzone",
  "Capricciosa Pizza",
  "Cheese Pizza",
  "Chicken Pizza",
  "Deep Dish Pizza",
  "Dominos Philly Cheese Steak Pizza",
  "Four Cheese Pizza",
  "Goat Cheese Pizza",
  "Grilled Pizza",
  "Hawaiian Pizza",
  "Margherita Pizza",
  "Mozzarella Pizza",
  "Mushroom Pizza",
  "Napoli Pizza",
  "New York Style Pizza",
  "Pepperoni Pizza",
  "Pizza Dough",
  "Pizza Hut Stuffed Crust Pizza",
  "Pizza Hut Supreme Pizza",
  "Pizza Rolls",
  "Quattro Formaggi Pizza",
  "Red Pepper Pizza",
  "Salami Pizza",
  "Sausage Pizza",
  "Seafood Pizza",
  "Shrimp Pizza",
  "Sicilian Pizza",
  "Spinach Feta Pizza",
  "Spinach Pizza",
  "Stuffed Crust Pizza",
  "Tarte Flambée",
  "Thin Crust Pizza",
  "Tuna Pizza",
  "Vegetable Pizza",
  "Vegetarian Pizza",
  "Veggie Pizza",
  "White Pizza",
  "Prickly Pear",
  "Arby’s Grand Turkey Club",
  "Arby’s Reuben",
  "Arby’s Roast Beef Classic",
  "Arby’s Roast Beef Max",
  "BBQ Rib",
  "Big N’ Tasty",
  "Burger King Angry Whopper",
  "Burger King Double Whopper",
  "Burger King Double Whopper with Cheese",
  "Burger King Original Chicken Sandwich",
  "Burger King Premium Alaskan Fish Sandwich",
  "Burger King Triple Whopper",
  "Burger King Whopper",
  "Burger King Whopper Jr.",
  "Burger King Whopper with Cheese",
  "Cheeseburger",
  "Chicken McNuggets",
  "Chicken Nuggets",
  "Chicken Pizziola",
  "Chicken Teriyaki Sandwich",
  "Chop Suey",
  "Curly Fries",
  "Double Cheeseburger",
  "Fish Sandwich",
  "Grilled Chicken Salad",
  "Italian BMT",
  "McDonald’s Big Mac",
  "McDonald’s Cheeseburger",
  "McDonald’s Chicken Nuggets",
  "McDonald’s Double Cheeseburger",
  "McDonald’s Filet-o-Fish",
  "McDonald’s McChicken",
  "McDonald’s McDouble",
  "McDonald’s McMuffi Egg",
  "McDonald’s McRib",
  "McDonald’s Mighty Wings",
  "McRib",
  "Nachos with Cheese",
  "Smoked Salmon",
  "Spicy Italian",
  "Subway Club Sandwich",
  "Tortilla Wrap",
  "Veggie Burger",
  "Veggie Delight",
  "Veggie Patty",
  "Wendy’s Baconator",
  "Wendy’s Jr. Bacon Cheeseburger",
  "Wendy’s Jr. Cheeseburger",
  "Wendy’s Son of Baconator",
  "Zinger",
  "Zinger Burger"
]

# filter CSV
mapping_csv.drop(mapping_csv[mapping_csv["Food"].isin(food_class_blacklist)].index, inplace=True)

Let's see the amount of classes we are left with:

In [8]:
# each food type is a row
# thus amount of rows is indicative of the amount of possible classes
CLASSES_AMOUNT = mapping_csv.shape[0]
print(CLASSES_AMOUNT)

322


Now that we have the filtered CSV we need, we can start downloading our data. The following automation queries Unsplash image search API for images and stores them in appropriate directories.

The initial run of the script will take a while because of bandwidth, latency and rate limiting issues. A free Unsplash account is allowed to query search API 50 times an hour - meaning 50 searches, each one with 30 results (images). So, in an hour, we can only download 30 * 50 = 3500 images. Given that we are dealing with a large amount of classes and each class has to have a proper amount of images - the script can take hours just because if the rate limiting alone. Additionally, if the notebook is being executed on a local machine, the bandwidth + latency issues come into play. These can be solved by running the script through Google Colab since a virtual machine is being provisioned for exectution which is located in a Data Center. Chances are, this virtual machine is far closer to Unsplash servers. 

The script skips download of a class if a directory for that class is already present, therefore a second run barely takes time since it does not query the API or download anything.

In [9]:
import logging
import time
import requests
import json
import urllib.request

RATE_LIMIT_TIMEOUT_SECONDS = 3720
IMAGE_FORMAT = "jpg"
PAGES_PER_CLASS = 3 # each page gets you 30 images

def query_unsplash_api(url, params, client_id):

    logging.info(f"qeurying API at {url} with params {params} ...")

    rate_limit_reached = False
    response_received = False

    # query api until a response is received
    while not response_received:

      try:

          # query api
          response = requests.get(url, params=params, timeout=60, headers={
              "Accept-Version": "v1",
              "Authorization": f"Client-ID {client_id}"
          })
          response.raise_for_status()

      except requests.HTTPError as e:

          # 403 - rate limit reached
          if e.response.status_code == 403:
            rate_limit_reached = True
          else:
            raise e

      # if HTTP status code is valid - response received
      else:
          response_received = True

      # handle rate limit reached
      if rate_limit_reached or ( response_received and int(response.headers["X-Ratelimit-Remaining"]) == 0 ):
          logging.info(f"rate limit reached, sleeping for {RATE_LIMIT_TIMEOUT_SECONDS} seconds...")
          time.sleep(RATE_LIMIT_TIMEOUT_SECONDS)

    # log remaining rate limit
    logging.info(f"X-Ratelimit-Remaining: {response.headers['X-Ratelimit-Remaining']}")

    return response

def main():

    # parse arguments
    CLIENT_ID = getpass.getpass("unsplash.com client ID: ")

    # set-up logger
    logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(message)s')

    # for each food
    for _, row in mapping_csv.iterrows():

        # get food and serving fields
        food = row['Food'].strip()
        food_dir_path = f"{DATA_DIR}/{food}"

        # download images only if directory is not yet present
        if os.path.exists(food_dir_path):
            
            logging.info(f"'{food_dir_path}' already exists, skipping")

        else:

            # initiate food directory
            os.makedirs(food_dir_path)

            # iterate result pages
            for page_index in range(PAGES_PER_CLASS):

              # query api for images
              search_response = json.loads(query_unsplash_api("https://api.unsplash.com/search/photos", params={
                  "query": f"{food}",
                  "page": f"{page_index + 1}",
                  "per_page": "30"
              }, client_id=CLIENT_ID).text)

              # iterate search results
              for image in search_response["results"]:

                  # download image
                  image_url = f'{image["urls"]["raw"]}&crop=entropy&w={IMG_SIZE[0]}&h={IMG_SIZE[1]}&fm={IMAGE_FORMAT}'
                  logging.info(f"downloading image for '{food}' at {image_url} ...")
                  urllib.request.urlretrieve(image_url, f"{food_dir_path}/{image['id']}.{IMAGE_FORMAT}")

if __name__ == "__main__":
    main()


unsplash.com client ID: ··········


[2022-06-01 14:06:58,600] qeurying API at https://api.unsplash.com/search/photos with params {'query': 'Artichoke', 'page': '1', 'per_page': '30'} ...
[2022-06-01 14:06:58,962] X-Ratelimit-Remaining: 46
[2022-06-01 14:06:59,776] downloading image for 'Artichoke' at https://images.unsplash.com/photo-1452715949016-e9a77a69c7d4?ixid=MnwzMjgyMjd8MHwxfHNlYXJjaHwxfHxBcnRpY2hva2V8ZW58MHx8fHwxNjU0MDkyNDE4&ixlib=rb-1.2.1&crop=entropy&w=299&h=299&fm=jpg ...
[2022-06-01 14:06:59,989] downloading image for 'Artichoke' at https://images.unsplash.com/photo-1524648881493-828393fe7890?ixid=MnwzMjgyMjd8MHwxfHNlYXJjaHwyfHxBcnRpY2hva2V8ZW58MHx8fHwxNjU0MDkyNDE4&ixlib=rb-1.2.1&crop=entropy&w=299&h=299&fm=jpg ...
[2022-06-01 14:07:00,116] downloading image for 'Artichoke' at https://images.unsplash.com/photo-1505613605686-52c838fbf402?ixid=MnwzMjgyMjd8MHwxfHNlYXJjaHwzfHxBcnRpY2hva2V8ZW58MHx8fHwxNjU0MDkyNDE4&ixlib=rb-1.2.1&crop=entropy&w=299&h=299&fm=jpg ...
[2022-06-01 14:07:00,251] downloading image for 'A

KeyboardInterrupt: ignored

Now that the data is present - we can begin construction of our network!

In [8]:
# read entire dataset and split to train and validation sets
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
training_set = train_gen.flow_from_directory( DATA_DIR,
                                              target_size=IMG_SIZE,
                                              batch_size=16,
                                              shuffle=True,
                                              class_mode='categorical',
                                              subset="training")
validation_set = train_gen.flow_from_directory( DATA_DIR,
                                                target_size=IMG_SIZE,
                                                batch_size=16,
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset="validation")

Mounted at /mnt/drive
Found 167 images belonging to 7 classes.
Found 41 images belonging to 7 classes.


# Base model setup

In [9]:
# use InceptionResNet2 as base model
base_model_function = tf.keras.applications.InceptionResNetV2
base_model_preprocess_input = tf.keras.applications.inception_resnet_v2.preprocess_input
base_model_decode_predictions = tf.keras.applications.inception_resnet_v2.decode_predictions

### Building the CNN

TODO

In [10]:
# init base layer with augmentations
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.5),
])

In [11]:
# init base model
base_model = base_model_function(   input_shape=IMG_SHAPE,
                                    include_top=False,
                                    weights='imagenet')

# freeze base layers
base_model.trainable = False

# show summary of our basic model
base_model.summary()

219070464/219055592 [==============================] - 1s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)  

In [12]:
# convert to vectors for feature extraction 
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# dense layer of unit per class, the highest one wins
prediction_layer = tf.keras.layers.Dense(units=CLASSES_AMOUNT, activation=tf.keras.activations.softmax)

# build model 
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = base_model_preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

# show summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 299, 299, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 299, 299, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 299, 299, 3)      0         
 a)                                                              
                                                                 
 inception_resnet_v2 (Functi  (None, 8, 8, 1536)       54336736  
 onal)                                                           
                                                             

### Training the model

TODO

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
history = model.fit(training_set, epochs=10, validation_data=validation_set)

Epoch 1/10


InvalidArgumentError: ignored

### Visualization of Loss and Accuracy

In [ ]:
# loss visualization
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch No.')
plt.legend(['Training Loss', 'Valuation Loss'], loc='upper right')
plt.show()

In [ ]:
# accuracy visualization
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch No.')
plt.legend(['Training Acc.', 'Valuation Acc.'], loc='lower right')
plt.show()

### Model evaluation

TODO